<a href="https://colab.research.google.com/github/mayurjadhav2002/ZENITH/blob/main/nlp_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [42]:
import numpy as np
import nltk
import string
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
import warnings
warnings.filterwarnings("ignore")

# Data Exploration and Cleaning

In [44]:
data=pd.read_csv('/content/WikiQA-train.tsv',sep='\t')


In [45]:
data.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20347 entries, 0 to 20346
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   QuestionID     20347 non-null  object
 1   Question       20347 non-null  object
 2   DocumentID     20347 non-null  object
 3   DocumentTitle  20347 non-null  object
 4   SentenceID     20347 non-null  object
 5   Sentence       20347 non-null  object
 6   Label          20347 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


In [47]:
data.isnull().sum()

QuestionID       0
Question         0
DocumentID       0
DocumentTitle    0
SentenceID       0
Sentence         0
Label            0
dtype: int64

# Training Dataset
as we see in our dataset that multiple rows have same QuestionId and DocumentID. this rows exhibit same data, grouping these rows will make tokenization less complex.

In order to train dataset create a new txt file.

* this *.txt file will consists of question and Sentence 



In [48]:
text_file = open("/content/text2.txt", "w")
prev_index = '0'
for index, row in data.iterrows():
  if row['QuestionID']== prev_index:
      text_file.write(row['Sentence'])
      prev_index = row['QuestionID']
  else:
      text_file.write('\n')
      text_file.write(row['Sentence'])
      prev_index = row['QuestionID']
text_file.close()

In [49]:
f = open('/content/text2.txt', 'r', errors='ignore')
raw_doc = f.read()
raw_doc = raw_doc.lower()


The punkt library is a pre-trained machine learning model that is used to perform sentence boundary detection, which is the process of identifying the boundaries between sentences in a piece of text. Sentence boundary detection is important for NLP tasks such as text classification, summarization, and translation, as it helps to identify the structure of the text and determine the meaning of individual sentences.

The wordnet library is a lexical database for the English language that is used to perform tasks such as synonym identification, antonym identification, and word sense disambiguation. It is based on the WordNet database, which is a large collection of English words and their relationships, organized into synonym sets (synsets). WordNet is widely used in NLP tasks such as text classification, information retrieval, and machine translation.

In summary, the punkt and wordnet libraries are useful tools for NLP projects because they provide pre-trained models and resources for performing common NLP tasks, such as sentence boundary detection and word sense disambiguation.

In [50]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [51]:
# Tokenization
sent_token = nltk.sent_tokenize(raw_doc)
word_token = nltk.word_tokenize(raw_doc)

In [52]:
lemmer = nltk.stem.WordNetLemmatizer()
remove_punc = dict((ord(punct),None) for punct in string.punctuation)
def lemmertoken(token):
  return [lemmer.lemmatize(i) for i in token]
def LemNormalize(text):
  return lemmertoken(nltk.word_tokenize(text.lower().translate(remove_punc)))

In [53]:
# Greet First
GREET_IN = ('hii', 'hello', 'hey','hi', 'chatbot', 'holla','how are you')
GREET_OUT = ['hey', 'hi', 'hi i am a chatbot', 'hii, nice to see you', 'hey you got something to ask', 'how you doing? ohh i asked, i am intelligent']
def greet(sentence):
  for word in sentence.split():
    if word.lower() in GREET_IN:
      return random.choice(GREET_OUT)


In [54]:
def response(user_input):
  robot_response = ''
  Fidfvect = TfidfVectorizer(tokenizer=LemNormalize,stop_words='english' )
  tfidfv = Fidfvect.fit_transform(sent_token)
  vals = cosine_similarity(tfidfv[-1], tfidfv)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if req_tfidf == 0:
    robot_response = robot_response + "Sorry, I don't understand"
    return robot_response 
  else:
    robot_response = robot_response + sent_token[idx]
    return robot_response 

In [56]:
start = True
print("Zenith Started")
print('----------------------------------')
while start:
  user_response = input("User: ").lower()
  try:
      if user_response != 'bye':
        if user_response =='thanks' or user_response=='thank you':
            start = False
            print("Zenith: You are Welcome!")
        else:
            if greet(user_response) != None:
              print('Zenith: ' + greet(user_response))
            else:
              sent_token.append(user_response)
              word_token = word_token + nltk.word_tokenize(user_response)
              final_words = list(set(word_token))
              print('Zenith: ', end=' ')
              print(response(user_response))
              sent_token.remove(user_response)
      else:
          start = False
          print('Zenith: Bye! Have a Good Day!')
  except:
      print('Zenith: please try again')

Zenith Started
----------------------------------
User: hii
Zenith: hi
User: how much water is in one spoon
Zenith:  this tablespoon has a capacity of about 15 ml.measuring spoonsin the us and parts of canada, a tablespoon is the largest type of spoon used for eating from a bowl.in the uk, europe and most commonwealth countries, a tablespoon is a type of large spoon usually used for serving.in countries where a tablespoon is a serving spoon, the nearest equivalent to the us tablespoon is either the dessert spoon or the soup spoon .a tablespoonful, nominally the capacity of one tablespoon, is commonly used as a measure of volume in cooking .it is abbreviated as t, tb, tbs, tbsp, tblsp, or tblspn.the capacity of ordinary tablespoons is not regulated by law and is subject to considerable variation.in the usa one tablespoon (measurement unit) is approximately 15 ml; the capacity of an actual tablespoon (dining utensil) ranges from 7 ml to 14 ml.in australia one tablespoon (measurement unit